In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2

In [ ]:
def preprocess_data(sample):
    image = tf.image.resize(sample['image'], (128, 128)) / 255.0
    mask = tf.image.resize(sample['segmentation_mask'], (128, 128))
    mask = tf.cast(mask > 0, tf.float32)
    return image, mask

In [ ]:
def data_augmentation(image, mask):
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
    return image, mask


In [ ]:
dataset, info = tfds.load('oxford_iiit_pet', with_info=True, split='train', data_dir='./data')
dataset = dataset.map(preprocess_data).map(data_augmentation).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:

def unet_model(input_size=(128, 128, 3)):
    base_model = MobileNetV2(input_shape=input_size, include_top=False)
    base_model.trainable = False

    inputs = layers.Input(input_size)
    x = base_model(inputs, training=False)


    x = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.fit(dataset, epochs=10)

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 44s 122ms/step - accuracy: 0.9529 - loss: 0.1053
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 1.0000 - loss: 5.3315e-25
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 19s 165ms/step - accuracy: 1.0000 - loss: 4.2562e-25
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 14s 117ms/step - accuracy: 1.0000 - loss: 6.7061e-24
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 22s 127ms/step - accuracy: 1.0000 - loss: 3.0993e-25
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 13s 114ms/step - accuracy: 1.0000 - loss: 7.0076e-24
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 19s 102ms/step - accuracy: 1.0000 - loss: 6.9647e-24
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 102ms/step - accuracy: 1.0000 - loss: 6.7679e-24
Epoch 9/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 1.0000 - loss: 1.9156e-25
Epoch 10/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 21s 102ms/step - accuracy: 1.0000 - loss: 6.7660e-24


In [ ]:
def iou_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersection = tf.reduce_sum(tf.abs(y_true * y_pred))
    sum_ = tf.reduce_sum(tf.abs(y_true) + tf.abs(y_pred))
    iou = intersection / (sum_ - intersection)
    return iou

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[iou_metric])
3

3

In [ ]:
results = model.evaluate(dataset)
print(f"Model Performance: {results}")

115/115 ━━━━━━━━━━━━━━━━━━━━ 15s 91ms/step - iou_metric: 1.0000 - loss: 3.3233e-25
Model Performance: [4.172508673957722e-25, 1.0]
